# Jupyter | Hub

## Spark e AWS Glue

### Documentações

- [Gerenciar partições para saída de ETL no AWS Glue](https://docs.aws.amazon.com/pt_br/glue/latest/dg/aws-glue-programming-etl-partitions.html)
- [Program AWS Glue ETL scripts in PySpark](https://docs.aws.amazon.com/glue/latest/dg/aws-glue-programming-python.html)
- [Getting started with Apache Spark on Amazon Athena](https://docs.aws.amazon.com/athena/latest/ug/notebooks-spark-getting-started.html)

## Setup do ambiente de desenvolvimento

- [AWS Glue Docker Image | Dev](https://aws.amazon.com/pt/blogs/big-data/develop-and-test-aws-glue-version-3-0-jobs-locally-using-a-docker-container/)

### Último comando utilizado 
    
        docker run -it -v ~/.aws:/home/glue_user/.aws -v $JUPYTER_WORKSPACE_LOCATION:/home/glue_user/workspace/jupyter_workspace/ -e AWS_PROFILE=default -e DISABLE_SSL=true -p 8888:8888 --name glue_jupyter_lab amazon/aws-glue-libs:glue_libs_4.0.0_image_01 /home/glue_user/jupyter/jupyter_start.sh


### Comando Docker | Prod

        docker run -itd -p <port_on_host>:<port_on_container_either_8888_or_8080> -p 4040:4040 -v ~/.aws:/home/glue_user/.aws -v <DiretorioOndeVaiSerArmazenadoOsNotebooks>:/home/glue_user/workspace/jupyter_workspace> -e AWS_PROFILE=<credential_setup_to_access_AWS_resources> --name <container_name> amazon/aws-glue-libs:glue_libs_4.0.0_image_01 <command_to_start_notebook_server>
    
- `-v ~/.aws:</root/.aws:rw>`: setar permissão de ler e escrever daquele diretório

### Comando Docker | Dev Spark UI

    docker run -it -v <ArquivosConfigAwsCLI>:/home/glue_user/.aws -v <DiretorioOndeVaiSerArmazenadoOsNotebooks>:/home/glue_user/workspace/jupyter_workspace/ -e AWS_PROFILE=<NomePerfilAWS> -e DISABLE_SSL=true --rm -p 4040:4040 -p 18080:18080 -p 8998:8998 -p 8888:8888 --name glue_jupyter_lab amazon/aws-glue-libs:aws-glue-libs:glue_libs_4.0.0_image_01 /home/glue_user/jupyter/jupyter_start.sh



## Executando scripts 

### Job Local | Pyspark

- O script para este caso deve ser executado dentro do diretório do projeto, devendo ter o Spark previamente instalado na sua máquina. 

#### Comandos utilizados

- Run: `spark-submit --master local caminho/para/seu-script.py argumento1 argumento2`
- Log: será apresentado logo em seguida.

### Job Local | AWS GLue

- O script deve ser executado no terminal do container Docker, com as credenciais AWS já configuradas. 

#### Comandos utilizados

- Run: `aws glue start-job-run --job-name SEU_NOME_DE_JOB`
- Log (json): `aws glue get-job-run --job-name MeuJobGlue --run-id SEU_ID_DE_EXECUCAO`

In [ ]:
import os # Sistema operacional 
import sys # Funções e variáveis da shell do Python
import getopt # Criação de opções de entrada pela CLI

# Spark
import pyspark  
from pyspark import SparkContext 
from pyspark.sql import SparkSession 
from pyspark.sql.functions import * 

# SparkSession
spark = SparkSession.builder.appName("App_1").getOrCreate()

# SparkContext
#sc = SparkContext()

# Job
if __name__ == "__main__":
    
    input_path = "/Users/camilabudke/Desenvolvimento/jupyter/dados_brutos/nomes.csv"
    target_path = "/Users/camilabudke/Desenvolvimento/jupyter/dados_processados/nomes"
    
    esquema = "nome STRING, sexo STRING, total INT, ano INT"
    
    nomes = spark.read.csv(input_path, header=True, schema=esquema)
    #nomes = spark.read.csv(sys.argv[1:], header=True, schema=esquema)
    
    
    # Transformação: Contagem das linhas do DataFrame
    linhas = nomes.count()
    
    # 1. Colocar os valores da coluna 'nome' em Maiúsculo
    from pyspark.sql.functions import upper
    nome_maiusc_col = nomes.withColumn("nome", upper(nomes["nome"]))
    
    # Output
    print("\n")
    print("\n Número de linhas: ", linhas)
    print("\n")
    
    #nome_maiusc_col.write.format("console").save()
    
    # Salvar a transformação particionada por sexo e ano
    # nome_maiusc_col.write.partitionBy("sexo", "ano").json(target_path)
    nome_maiusc_col.write.partitionBy("sexo", "ano").mode('overwrite').json(target_path)

    # Fim da aplicação
    spark.stop()

### Extraindo os dados

In [ ]:
# Sem API
caminho_texto = "./README.md"
caminho_csv = "./nomes.csv"
#nomes = spark.read.format("csv").load(caminho_csv)

# Com API


## Jobs

- [Conta palavras de um arquivo de texto | Job 1](lab/jobs/job_conta_palavras.ipynb)
- [Colunas em maiúsculo particionadas | Job 2](lab/jobs/glue_lab.ipynb)
- []()